In [1]:
from cosmikyu import gan, config, datasets, transforms, utils
from cosmikyu import nn as cnn
import os
import torch
import mlflow
from orphics import maps
import torchsummary
import matplotlib.pyplot as plt
import torchvision
from skimage.measure import block_reduce
from skimage import io
import torchsummary

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
sample_interval = 200
save_interval = 5
batch_size = 8
nin_ch = 3
nout_ch = 3
nepochs = 100
cuda = True

NRGB = transforms.NormalizeRGB()
RESIZE = transforms.BlockReduce((1,2,2))
CROP = transforms.Crop((256,256))

maps_dir = "/home/dwhan89/workspace/cosmikyu/data/maps"
MDB = datasets.MapDataSet(maps_dir, data_type="train", transforms=[RESIZE, CROP, NRGB], dummy_label=True)
shape = MDB[0][0].shape

/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
dataloader = torch.utils.data.DataLoader(
    MDB,
    batch_size=batch_size,
    shuffle=True,
)

In [4]:
STanh = cnn.ScaledTanh(1., 1)
PIXGAN = gan.PIXGAN("sehgal_pixgan_251020", shape, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=cuda, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=4, kernal_size=5, stride=2,
                          padding=2, output_padding=1, gen_act=STanh, nin_channel=3, nout_channel=3, nthresh_layer_gen=4, nthresh_layer_disc=3)

torchsummary.summary(PIXGAN.discriminator, shape, device="cuda")
torchsummary.summary(PIXGAN.generator, (3,256,256), device="cuda")

Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,664
         LeakyReLU-2         [-1, 64, 128, 128]               0
            Conv2d-3          [-1, 128, 64, 64]         204,928
       BatchNorm2d-4          [-1, 128, 64, 64]             256
         LeakyReLU-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 256, 32, 32]         819,456
       BatchNorm2d-7          [-1, 256, 32, 32]             512
         LeakyReLU-8          [-1, 256, 32, 32]               0
            Conv2d-9          [-1, 512, 16, 16]       3,277,312
      BatchNorm2d-10          [-1, 512, 16, 16]           1,024
        LeakyReLU-11          [-1, 512, 16, 16]               0
           Conv2d-12            [-1, 512, 8, 8]       6,554,112
      BatchNorm2d-13            [-1, 512, 8, 8]           1,024
        LeakyReLU-14            [-1, 51

In [ ]:
mlflow.set_experiment(PIXGAN.identifier)
with mlflow.start_run(experiment_id=PIXGAN.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    PIXGAN.train(
        dataloader,
        nepochs=nepochs,
        ncritics=1,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=1e-4,
        betas=(0.5, 0.9),
        lambda_l1=10.
    )


/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.M

loading saved states 
failed to load saved states
saving states _0
[Epoch 0/100] [Batch 0/137] [D loss: 0.564923] [G loss: 7.998635]
[Epoch 0/100] [Batch 1/137] [D loss: 0.388171] [G loss: 5.734461]
[Epoch 0/100] [Batch 2/137] [D loss: 0.315446] [G loss: 6.585621]
[Epoch 0/100] [Batch 3/137] [D loss: 0.241607] [G loss: 7.049995]
[Epoch 0/100] [Batch 4/137] [D loss: 0.129123] [G loss: 7.701163]
[Epoch 0/100] [Batch 5/137] [D loss: 0.102165] [G loss: 6.908024]
[Epoch 0/100] [Batch 6/137] [D loss: 0.059471] [G loss: 7.205299]
[Epoch 0/100] [Batch 7/137] [D loss: 0.072559] [G loss: 8.216427]
[Epoch 0/100] [Batch 8/137] [D loss: 0.042788] [G loss: 7.967537]
[Epoch 0/100] [Batch 9/137] [D loss: 0.054079] [G loss: 8.488924]
[Epoch 0/100] [Batch 10/137] [D loss: 0.022103] [G loss: 8.390393]
[Epoch 0/100] [Batch 11/137] [D loss: 0.040841] [G loss: 6.823417]
[Epoch 0/100] [Batch 12/137] [D loss: 0.070474] [G loss: 9.810135]
[Epoch 0/100] [Batch 13/137] [D loss: 0.030115] [G loss: 8.608964]
[Epoc